In [1]:
%matplotlib widget
import matplotlib.pyplot as plt
import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
import pandas as pd
from shapely.geometry import LineString
import datetime
import numpy as np
from pyproj import Transformer
from pyproj import CRS
from shapely import MultiPoint
from pyproj.aoi import AreaOfInterest
from pyproj.database import query_utm_crs_info

Clip Spatial

In [2]:
cycle = 14

kml_file = '/home/jovyan/shared-public/ICESat-2-Hackweek/ground_tracks/cycle%02i_1hz_points.pkl' % cycle
search_file = 'search_circle_ll.geojson'

geo_kml = pd.read_pickle(kml_file)
geo_search = gpd.GeoDataFrame.from_file(search_file)

# Clip gdf 'geo_kml' of kmls with gdf 'geo_search' of AOI (circle polygon)
geo_clipped_df = gpd.clip(geo_kml, geo_search).copy()
geo_clipped_df.sort_values(by='timestamp', inplace=True)
geo_clipped_df.reset_index(inplace=True, drop=True)

In [3]:
geo_clipped = geo_clipped_df.copy()

In [4]:
# add linestrings to the file
linestrings = geo_clipped.groupby(['rgt'])['lat','lon'].apply(lambda x: LineString(list(zip(x.lon.tolist(),x.lat.tolist()))))
times = geo_clipped.groupby(by='rgt')['timestamp'].mean()
dflines = gpd.GeoDataFrame(times, geometry=linestrings, crs="EPSG:4326")
dflines['time_str'] = [datetime.datetime.strftime(datetime.datetime.fromtimestamp(t), '%a %Y-%b-%d %H:%M:%S') for t in dflines.timestamp]
dflines.reset_index(inplace=True)

# add the initial search area
geo = geo_search.copy()
geo['rgt'] = 'none'
geo['timestamp'] = 0
geo['time_str'] = 'none'

# put all of the dataframes together
df_out = pd.concat((geo, dflines, geo_clipped[dflines.keys()]))
df_out.reset_index(inplace=True, drop=True)
df_out.explore()

/tmp/ipykernel_1556/2568753503.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  linestrings = geo_clipped.groupby(['rgt'])['lat','lon'].apply(lambda x: LineString(list(zip(x.lon.tolist(),x.lat.tolist()))))


In [5]:
geo_clipped = geo_clipped_df.copy()

crs_lonlat = CRS('epsg:4326')

distances = [-3345, -3255, -45, 45, 3255, 3345]
spots = ['1l', '1r', '2l', '2r', '3l', '3r']

for rgt in np.unique(geo_clipped.rgt):
    
    df_rgt = geo_clipped[geo_clipped.rgt == rgt]
    n_pts = len(df_rgt)
    
    for i in range(n_pts):
        
        # get the locally centered CRS from this point
        thispt = df_rgt.iloc[i]
        cust = CRS("+proj=tmerc +lat_0={0} +lon_0={1} +datum=WGS84 +units=m".format(thispt.lat, thispt.lon))
        
        # get the surrounding points and transform
        edges = np.clip(np.array([i-1, i+1]), 0, n_pts-1)
        thisdata = df_rgt.iloc[edges[0]:edges[1]+1,:]
        
        xydata = thisdata.to_crs(cust)
        
        dx = xydata.iloc[-1].geometry.x - xydata.iloc[0].geometry.x
        dy = xydata.iloc[-1].geometry.y - xydata.iloc[0].geometry.y
        normal = np.array([-dy, dx])
        unit_normal = normal / np.linalg.norm(normal)
        
        xs, ys = [0.0], [0.0]
        for d in distances:
            xy = d * unit_normal
            xs.append(xy[0])
            ys.append(xy[1])
            
        thisdf = df_rgt.iloc[[i]].to_crs(cust)
        thisdf['geometry'] = MultiPoint(np.transpose(np.array([xs,ys])))
        resultdf = thisdf.to_crs('epsg:4326')
        geo_clipped.loc[thispt.name,'geometry'] = resultdf.iloc[0].geometry


df_ex = pd.concat((df_out, geo_clipped[df_out.keys()]))

filename_out = 'test_beams.geojson'
df_ex.to_file(filename_out, driver="GeoJSON")

df_ex.iloc[1:,:].explore()

 

In [6]:
# TODO: make linestrings from all the beam points
# beam_strings = geo_clipped.groupby(['rgt'])['geometry'].apply(lambda x: LineString(list(zip(x.lon.tolist(),x.lat.tolist()))))

In [ ]:
spots = ['1l', '1r', '2l', '2r', '3l', '3r']
columns = np.array([[s+'_lon', s+'_lat'] for s in spots]).flatten()
geo_clipped[columns] = np.nan

for i in range(len(geo_clipped)):
    x = geo_clipped.iloc[i]
    pts = np.array([(x.geometry.geoms[j].x, x.geometry.geoms[j].y) for j in range(len(x.geometry.geoms))]).flatten()
    geo_clipped.loc[x.name, columns] = pts[2:]

In [ ]:
fig, ax = plt.subplots(figsize=[9,9])
for j, rgt in enumerate(np.unique(geo_clipped.rgt)):
    df_rgt = geo_clipped[geo_clipped.rgt == rgt]
    ax.plot(df_rgt.lon,df_rgt.lat,'k-')
    for i,s in enumerate(spots):
        ax.plot(df_rgt['%s_lon'%s],df_rgt['%s_lat'%s],marker='o',lw=0.5,ms=3,c='C%i'%j)

In [ ]:
from pyproj.aoi import AreaOfInterest
from pyproj.database import query_utm_crs_info

geo_clipped = geo_clipped_df.copy()
#geo_clipped = geo_clipped[geo_clipped.rgt == geo_clipped.rgt.iloc[0]].copy()

crs_lonlat = CRS('epsg:4326')

distances = [-3345, -3255, -45, 45, 3255, 3345]
spots = ['1l', '1r', '2l', '2r', '3l', '3r']
for s in spots:
    for l in ['lon', 'lat']:
        geo_clipped['%s_%s' % (s,l)] = np.nan

for rgt in np.unique(geo_clipped.rgt):
    
    df_rgt = geo_clipped[geo_clipped.rgt == rgt]
    n_pts = len(df_rgt)
    
    for i in range(n_pts):
        
        thispt = df_rgt.iloc[i]
        # cust = CRS("+proj=tmerc +lat_0={0} +lon_0={1} +datum=WGS84 +units=m".format(thispt.lat, thispt.lon))
        # cust = CRS('+proj=tmer +lat_0=%.5f +lon_0=%.5f +k=1 +x_0=0 +y_0=0 +ellps=WGS84 +towgs84=0,0,0,0,0,0,0 +units=m +no_defs' % (thispt.lat, thispt.lon))
        # cust = CRS('+proj=tmerc +lat_0={0} +lon_0={1} +k=1 +x_0=0 +y_0=0 +ellps=WGS84 +towgs84=0,0,0,0,0,0,0 +units=m +no_defs'.format(thispt.lat, thispt.lon))
        # utm_crs_list = query_utm_crs_info(datum_name="WGS 84",area_of_interest=AreaOfInterest(
        #         west_lon_degree=thispt.lon,south_lat_degree=thispt.lat,east_lon_degree=thispt.lon,north_lat_degree=thispt.lat,),)
        # cust = CRS.from_epsg(utm_crs_list[0].code)
        cust = CRS("+proj=tmerc +lat_0={0} +lon_0={1} +datum=WGS84 +units=m".format(thispt.lat, thispt.lon))
        # cust = CRS("EPSG:3995")
        
        lonlat2xy = Transformer.from_crs(crs_lonlat, cust)
        xy2lonlat = Transformer.from_crs(cust, crs_lonlat)
        
        edges = np.clip(np.array([i-1, i+1]), 0, n_pts-1)
        pt0 = df_rgt.iloc[edges[0]]
        pt1 = df_rgt.iloc[edges[1]]
        
        x, y = lonlat2xy.transform(thispt.lon, thispt.lat)
        x0, y0 = lonlat2xy.transform(pt0.lon, pt0.lat)
        x1, y1 = lonlat2xy.transform(pt1.lon, pt1.lat)
        dx = x1 - x0
        dy = y1 - y0
        normal = np.array([-dy, dx])
        unit_normal = normal / np.linalg.norm(normal)
        print(rgt, edges, dx, dy, unit_normal)
        
        for i,d in enumerate(distances):
            x_beam, y_beam = np.array([x,y]) + d * unit_normal
            # print(d, unit_normal, d * unit_normal, x_beam, y_beam)
            lon_beam, lat_beam = xy2lonlat.transform(x_beam, y_beam)
            geo_clipped.loc[thispt.name, '%s_lon' % spots[i]] = lon_beam
            geo_clipped.loc[thispt.name, '%s_lat' % spots[i]] = lat_beam

geo_clipped['geometry'] = geo_clipped.apply(lambda x: MultiPoint(np.array(x.iloc[6:]).reshape(-1,2)), axis=1)

df_ex = pd.concat((df_out, geo_clipped[df_out.keys()]))
# df_ex = geo_clipped

df_ex.iloc[1:,:].explore()

# filename_out = 'test_beams.geojson'
#df_ex.to_file(filename_out, driver="GeoJSON") 

In [ ]:
fig, ax = plt.subplots(figsize=[9,5])
for rgt in np.unique(geo_clipped.rgt):
    df_rgt = geo_clipped[geo_clipped.rgt == rgt]
    x, y = lonlat2xy.transform(df_rgt.lon, df_rgt.lat)
    ax.plot(x,y,'k-')
    # for i,s in enumerate(spots):
    #     ax.plot(df_rgt['%s_lon'%s],df_rgt['%s_lat'%s],marker='o',lw=0.5,ms=3,c='C%i'%i)
        
ax.set_aspect('equal')

In [ ]:
geo_clipped

In [ ]:
geo_clipped

In [ ]:
geo_clipped['1l_lon'], geo_clipped['1l_lat']

In [ ]:
from shapely import MultiPoint
r = geo_clipped.iloc[0]
geo_clipped['geometry'] = geo_clipped.apply(lambda x: MultiPoint(np.array(x.iloc[6:]).reshape(-1,2)), axis=1)

In [ ]:
geo_clipped

In [ ]:
len(geo_clipped_df)

In [ ]:
df_ex.iloc[1:,:]

In [ ]:
# add linestrings to the file
linestrings = geo_clipped.groupby(['rgt'])['lat','lon'].apply(lambda x: LineString(list(zip(x.lon.tolist(),x.lat.tolist()))))
times = geo_clipped.groupby(by='rgt')['timestamp'].mean()
dflines = gpd.GeoDataFrame(times, geometry=linestrings, crs="EPSG:4326")
dflines['time_str'] = [datetime.datetime.strftime(datetime.datetime.fromtimestamp(t), '%a %Y-%b-%d %H:%M:%S') for t in dflines.timestamp]
dflines.reset_index(inplace=True)

# add the initial search area
geo = geo_search.copy()
geo['rgt'] = 'none'
geo['timestamp'] = 0
geo['time_str'] = 'none'

# put all of the dataframes together
df_out = pd.concat((geo, dflines, geo_clipped[dflines.keys()]))
df_out.reset_index(inplace=True, drop=True)
df_out.explore()

In [ ]:
# save it all to an output file
filename_out = search_file[:search_file.rfind('.')] + '_results.geojson'
df_out.to_file(filename_out, driver="GeoJSON") 
print('Wrote results to %s!' % filename_out)
df_out